# Imports

In [1]:
import torchio as tio
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
import random

In [2]:
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Data Loader

In [3]:
root = Path("./Liver/imagesTr/")
sample_path = list(root.glob("liver*"))
random.seed(452)
random.shuffle(sample_path)
for i in sample_path:
    print(i)

Liver\imagesTr\liver_4.nii.gz
Liver\imagesTr\liver_64.nii.gz
Liver\imagesTr\liver_1.nii.gz
Liver\imagesTr\liver_28.nii.gz
Liver\imagesTr\liver_33.nii.gz
Liver\imagesTr\liver_127.nii.gz
Liver\imagesTr\liver_120.nii.gz
Liver\imagesTr\liver_62.nii.gz
Liver\imagesTr\liver_66.nii.gz
Liver\imagesTr\liver_89.nii.gz
Liver\imagesTr\liver_104.nii.gz
Liver\imagesTr\liver_24.nii.gz
Liver\imagesTr\liver_82.nii.gz
Liver\imagesTr\liver_83.nii.gz
Liver\imagesTr\liver_38.nii.gz
Liver\imagesTr\liver_102.nii.gz
Liver\imagesTr\liver_126.nii.gz
Liver\imagesTr\liver_75.nii.gz
Liver\imagesTr\liver_29.nii.gz
Liver\imagesTr\liver_69.nii.gz
Liver\imagesTr\liver_52.nii.gz
Liver\imagesTr\liver_73.nii.gz
Liver\imagesTr\liver_43.nii.gz
Liver\imagesTr\liver_68.nii.gz
Liver\imagesTr\liver_19.nii.gz
Liver\imagesTr\liver_63.nii.gz
Liver\imagesTr\liver_87.nii.gz
Liver\imagesTr\liver_58.nii.gz
Liver\imagesTr\liver_128.nii.gz
Liver\imagesTr\liver_78.nii.gz
Liver\imagesTr\liver_35.nii.gz
Liver\imagesTr\liver_113.nii.gz
Liv

In [4]:
def find_last_number_in_string(text):
    last_number = ""
    found_number = False

    for char in reversed(text):
        if char.isdigit():
            last_number = char + last_number
            found_number = True
        elif found_number:
            break

    if found_number:
        return int(last_number)
    else:
        return None


In [5]:
csv = pd.read_csv("./Book1.csv")

def find_index_by_hasta_num(dataframe, target_hasta_num):
    filtered_df = dataframe[dataframe['hasta_num'] == target_hasta_num]
    if not filtered_df.empty:
        return filtered_df.index[0]
    else:
        return None

In [6]:
y = []

for i in tqdm(range(123)):
    y.append(csv["tumor_durum"][find_index_by_hasta_num(csv,find_last_number_in_string(str(sample_path[i])))])

  0%|          | 0/123 [00:00<?, ?it/s]

In [7]:
j = 0
for i in sample_path:
    print(str(i) + "   " + str(y[j]))
    j+=1

Liver\imagesTr\liver_4.nii.gz   1
Liver\imagesTr\liver_64.nii.gz   1
Liver\imagesTr\liver_1.nii.gz   0
Liver\imagesTr\liver_28.nii.gz   1
Liver\imagesTr\liver_33.nii.gz   1
Liver\imagesTr\liver_127.nii.gz   0
Liver\imagesTr\liver_120.nii.gz   0
Liver\imagesTr\liver_62.nii.gz   0
Liver\imagesTr\liver_66.nii.gz   0
Liver\imagesTr\liver_89.nii.gz   0
Liver\imagesTr\liver_104.nii.gz   1
Liver\imagesTr\liver_24.nii.gz   0
Liver\imagesTr\liver_82.nii.gz   1
Liver\imagesTr\liver_83.nii.gz   0
Liver\imagesTr\liver_38.nii.gz   0
Liver\imagesTr\liver_102.nii.gz   0
Liver\imagesTr\liver_126.nii.gz   0
Liver\imagesTr\liver_75.nii.gz   0
Liver\imagesTr\liver_29.nii.gz   0
Liver\imagesTr\liver_69.nii.gz   0
Liver\imagesTr\liver_52.nii.gz   0
Liver\imagesTr\liver_73.nii.gz   0
Liver\imagesTr\liver_43.nii.gz   0
Liver\imagesTr\liver_68.nii.gz   0
Liver\imagesTr\liver_19.nii.gz   0
Liver\imagesTr\liver_63.nii.gz   0
Liver\imagesTr\liver_87.nii.gz   0
Liver\imagesTr\liver_58.nii.gz   0
Liver\imagesTr\li

In [8]:
Train = tio.Compose([
            tio.CropOrPad((256, 256, 70)),
            tio.RescaleIntensity((-1, 1)),
            ])

Test = tio.Compose([
            tio.CropOrPad((256, 256, 70)),
            tio.RescaleIntensity((-1, 1)),
            ])

# Test and Train Split

In [9]:
X1 = []
X2 = []
y_train = []
y_test = []
j = 0

for i in tqdm(sample_path):
    if (j < 25):
        tio_image = tio.ScalarImage(i)
        processed_image = Test(tio_image)
        np_processed_image = np.array(processed_image)
        X1.append(np_processed_image[0].reshape(-1))
        j+=1
    else:
        tio_image = tio.ScalarImage(i)
        processed_image = Train(tio_image)
        np_processed_image = np.array(processed_image)
        X2.append(np_processed_image[0].reshape(-1))
        j+=1

X_test = np.array(X1)
X_train = np.array(X2)
y_test, y_train = np.split(y, [25])
del X1, X2, j

  0%|          | 0/123 [00:00<?, ?it/s]

In [10]:
print(X_test.shape, X_train.shape)

(25, 4587520) (98, 4587520)


# Over Sampling

In [11]:
from imblearn.over_sampling import RandomOverSampler

# Oversample the minority class
ros = RandomOverSampler(random_state=0)
X_train, y_train = ros.fit_resample(X_train, y_train)
print(X_train.shape)

(144, 4587520)


# PCA

In [12]:
from sklearn.decomposition import PCA

pca = PCA(.95)
pca_train = pca.fit_transform(X_train)
pca_test = pca.transform(X_test)
print(pca_train.shape, pca_test.shape)

(144, 52) (25, 52)


# Train

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

def models(x, y):
    # Training of the Logistic Regression model
    log = LogisticRegression(solver='liblinear', penalty='l1',C= 1291.5496650148827, max_iter=1000, random_state=42)
    log.fit(x, y)
    
    # Training of the KNN model
    knn = KNeighborsClassifier(n_neighbors = 5, metric = "minkowski", p = 2)
    knn.fit(x, y)
    
    # Training of the Linear SVM model
    svm1 = LinearSVC(random_state = 42)
    svm1.fit(x, y)
    
    # Training of the RBF Kernel SVM model
    kernel_svm = SVC(max_iter=1000,kernel = "rbf", random_state = 42, C = 0.1)
    kernel_svm.fit(x, y)
    
    # Training of the Naive Bayes
    naive = GaussianNB()
    naive.fit(x, y)
    
    # Training of the Decision Tree model
    dtree = DecisionTreeClassifier(criterion = "gini", random_state = 42)
    dtree.fit(x, y)
    
    # Training of the Random Forest model
    rforest = RandomForestClassifier(max_depth=70, min_samples_split=4, min_samples_leaf=3, 
                                    n_estimators = 1000, criterion = "gini" ,max_features='sqrt',random_state = 42)
    rforest.fit(x, y)
    
    print('[0]Logistic Regression Training Accuracy:', log.score(x, y))
    print('[1]K Nearest Neighbor Training Accuracy:', knn.score(x, y))
    print('[2]Support Vector Machine (Linear Classifier) Training Accuracy:', svm1.score(x, y))
    print('[3]Support Vector Machine (RBF Classifier) Training Accuracy:', kernel_svm.score(x, y))
    print('[4]Gaussian Naive Bayes Training Accuracy:', naive.score(x, y))
    print('[5]Decision Tree Classifier Training Accuracy:', dtree.score(x, y))
    print('[6]Random Forest Classifier Training Accuracy:', rforest.score(x, y))
    
    return log, knn, svm1, kernel_svm, naive, dtree, rforest

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

model = models(pca_train, y_train)

C:\Users\furka\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[0]Logistic Regression Training Accuracy: 1.0
[1]K Nearest Neighbor Training Accuracy: 0.8333333333333334
[2]Support Vector Machine (Linear Classifier) Training Accuracy: 1.0
[3]Support Vector Machine (RBF Classifier) Training Accuracy: 0.7847222222222222
[4]Gaussian Naive Bayes Training Accuracy: 0.8888888888888888
[5]Decision Tree Classifier Training Accuracy: 1.0
[6]Random Forest Classifier Training Accuracy: 1.0


# Evaluation

In [15]:
for i in tqdm(range(len(model))):
    print(confusion_matrix(y_train, model[i].predict(pca_train)))
    print("Model[{}] Training Accuracy".format(i),accuracy_score(y_train, model[i].predict(pca_train)))
    print("Model[{}] Training Precision".format(i),precision_score(y_train, model[i].predict(pca_train)))
    print("Model[{}] Training Recall".format(i),recall_score(y_train, model[i].predict(pca_train)))  
    print()

  0%|          | 0/7 [00:00<?, ?it/s]

[[72  0]
 [ 0 72]]
Model[0] Training Accuracy 1.0
Model[0] Training Precision 1.0
Model[0] Training Recall 1.0

[[62 10]
 [14 58]]
Model[1] Training Accuracy 0.8333333333333334
Model[1] Training Precision 0.8529411764705882
Model[1] Training Recall 0.8055555555555556

[[72  0]
 [ 0 72]]
Model[2] Training Accuracy 1.0
Model[2] Training Precision 1.0
Model[2] Training Recall 1.0

[[60 12]
 [19 53]]
Model[3] Training Accuracy 0.7847222222222222
Model[3] Training Precision 0.8153846153846154
Model[3] Training Recall 0.7361111111111112

[[66  6]
 [10 62]]
Model[4] Training Accuracy 0.8888888888888888
Model[4] Training Precision 0.9117647058823529
Model[4] Training Recall 0.8611111111111112

[[72  0]
 [ 0 72]]
Model[5] Training Accuracy 1.0
Model[5] Training Precision 1.0
Model[5] Training Recall 1.0

[[72  0]
 [ 0 72]]
Model[6] Training Accuracy 1.0
Model[6] Training Precision 1.0
Model[6] Training Recall 1.0



In [16]:
for i in tqdm(range(len(model))):
    print(confusion_matrix(y_test, model[i].predict(pca_test)))
    print("Model[{}] Testing Accuracy".format(i),accuracy_score(y_test, model[i].predict(pca_test)))
    print("Model[{}] Testing Precision".format(i),precision_score(y_test, model[i].predict(pca_test)))
    print("Model[{}] Testing Recall".format(i),recall_score(y_test, model[i].predict(pca_test)))      
    print()

  0%|          | 0/7 [00:00<?, ?it/s]

[[ 3 16]
 [ 1  5]]
Model[0] Testing Accuracy 0.32
Model[0] Testing Precision 0.23809523809523808
Model[0] Testing Recall 0.8333333333333334

[[11  8]
 [ 2  4]]
Model[1] Testing Accuracy 0.6
Model[1] Testing Precision 0.3333333333333333
Model[1] Testing Recall 0.6666666666666666

[[ 1 18]
 [ 0  6]]
Model[2] Testing Accuracy 0.28
Model[2] Testing Precision 0.25
Model[2] Testing Recall 1.0

[[11  8]
 [ 3  3]]
Model[3] Testing Accuracy 0.56
Model[3] Testing Precision 0.2727272727272727
Model[3] Testing Recall 0.5

[[ 0 19]
 [ 0  6]]
Model[4] Testing Accuracy 0.24
Model[4] Testing Precision 0.24
Model[4] Testing Recall 1.0

[[ 7 12]
 [ 2  4]]
Model[5] Testing Accuracy 0.44
Model[5] Testing Precision 0.25
Model[5] Testing Recall 0.6666666666666666

[[ 7 12]
 [ 2  4]]
Model[6] Testing Accuracy 0.44
Model[6] Testing Precision 0.25
Model[6] Testing Recall 0.6666666666666666

